# Plantilla de desarrollo para primer examen parcial

**Pautas:**
- La presente plantilla es un EJEMPLO de cómo ordenar el código de tu examen
- Tienes la libertad DE AGREGAR todos los métodos y secciones en el examen que consideres necesarias
- Realizar el desarrollo por medio de métodos, por ejemplo, ReadInfo(), TrainModel(), etc 
- Los métodos deberán de estar lo mas claro y modularizados que sea posible
- Realizar la documentación de cada método por medio de comentarios y DocStrings 
- Deberás de utilizar un modelo de ML o algún ensamble de os mismos (SVC, DT, NB, KNN, etc)
- Recuerda que puedes usar un split de los datos para entrenamiento y validación
- Puedes revisar la documentación de Sklearn, o la librría que decidas utilizar para entender los parámetros de entrenamiento de los modelos
- NO está permitido el uso de modelos de Deep Learning (DNN, CNN, LSTM, etc.) NI el uso de embeddings

## Librerías a utilizar

In [132]:
# Librerías profe
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer

# Librerías actuales
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import classification_report

nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /home/eubgo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/eubgo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/eubgo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Lectura de Dataset

In [133]:
# Aquí cargamos la información del DataSet de entrenamiento
def read_corpus(path):
    """Este método lee los de datos del corpus y los pasa a un dataFrame

    Args:
        path (string): Ubicación del archivo de entrada (Corpus)
    """
    df = pd.read_csv(path)
    df.index = np.arange(1, len(df) + 1)
    print("Elementos en el DataSet:", len(df))
    return df

## Feature Engineering

In [134]:
tfidf_vect = TfidfVectorizer(ngram_range=(1, 2))
# Longitud: numero de palabras en texto
# Diversidad léxica: palabras únicas vs palabras totales en el texto

## Preprocesamiento

In [135]:
# Inserta lo que consideres necesario aquí, por ejemplo
def preprocess(df):
    """Método para preprocesar el texto

    Args:
        df (dataframe): Dataframe a aplicar transformaciones

    Returns:
        dataframe: Dataframe transformado
    """
    
    for column in df.columns:
        df[column] = df[column].str.lower()

    stop_words = stopwords.words("english")
    df["title"] = df["title"].apply(lambda x: " ".join([word for word in x.split() if word not in stop_words]))

    df["title"] = df["title"].str.replace("[^\w\s]", "")

    df["title"] = df["title"].apply(nltk.word_tokenize)

    lemmatizer = WordNetLemmatizer()

    df["title"] = df["title"].apply(lambda x: [lemmatizer.lemmatize(word) for word in x if not word.isdigit()])

    # df['text_length'] = df['title'].apply(len)  # Longitud del texto
    # df['lexical_diversity'] = df['title'].apply(lambda x: len(set(x)) / len(x))
    
    return df

## Data Augmentation

In [136]:
from nltk.corpus import wordnet
nltk.download('wordnet')
def get_synonyms(word):
    """Obtiene sinónimos de una palabra usando WordNet.

    Args:
        word (str): Palabra para la cual obtener sinónimos.

    Returns:
        list: Lista de sinónimos.
    """
    from nltk.corpus import wordnet
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return list(synonyms)

def replace_with_synonym(sentence):
    """Reemplaza palabras en una oración con un sinónimo aleatorio.

    Args:
        sentence (str): Oración a modificar.

    Returns:
        str: Oración modificada.
    """
    words = sentence.split()
    for i, word in enumerate(words):
        synonyms = get_synonyms(word)
        if synonyms:
            words[i] = synonyms[0].replace("_", " ")
    return ' '.join(words)

def augment(df):
    df_augmented = df.copy()
    clickbait_rows = df_augmented[df_augmented["label"] == "clickbait"]
    clickbait_rows["title"] = clickbait_rows["title"].apply(replace_with_synonym)
    df_combined = pd.concat([df, clickbait_rows], ignore_index=True)
    return df_combined



#probar augment
df = ReadCorpus("DataSet para entrenamiento del modelo.csv")
df = augment(df)
df = augment(df)

#count duplicate rows
duplicate_rows = df[df.duplicated()]
print("duplicate rows:", duplicate_rows)

df = Preprocess(df)

print(df.tail())

# contar numero de news y clickbait
news = df[df["label"] == "news"]
clickbait = df[df["label"] == "clickbait"]
print("news:", len(news))
print("clickbait:", len(clickbait))

[nltk_data] Downloading package wordnet to /home/eubgo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Elementos en el DataSet: 16823


/tmp/ipykernel_71873/4217138993.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clickbait_rows["title"] = clickbait_rows["title"].apply(replace_with_synonym)
/tmp/ipykernel_71873/4217138993.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clickbait_rows["title"] = clickbait_rows["title"].apply(replace_with_synonym)


duplicate rows:            label                                              title
540         news            The Situation RoomVerifizierter Account
1127        news            The Situation RoomVerifizierter Account
1674        news                                           Facebook
1783        news  Big Mac Makeover Helps McDonald's Overcome Res...
2117   clickbait  17 Limited-Edition Beauty Products You Need To...
...          ...                                                ...
26903  clickbait  ten subject you claim for to passion ahead the...
26907  clickbait  link up Hoosier State breast feeding inside se...
26911  clickbait           literally exactly XIX really gravid barf
27081  clickbait  ten subject you claim for to passion ahead the...
27088  clickbait  sextuplet low-sweat recitation to dumbfound th...

[3652 rows x 2 columns]
           label                                              title
27096  clickbait  [brokaw, :, ‘, democrat, got, dumbfound, antio...
27097  

## Data Reduction

## Entrenamiento del modelo

In [137]:
model = RandomForestClassifier()


def train_model(x, y):
    """Este método realiza el entrenamiento del modelo (Ejemplo)

    Args:
        x (list): Lista con los textos a transformar
        y (list): Lista con los valores de y (Salida)

    Returns:
        model: Modelo entrenado
    """
    # Parámetros para GridSearchCV
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10]
    }

    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(x, y)

    # Entrenando el modelo con los mejores parámetros
    best_model = grid_search.best_estimator_
    best_model.fit(x, y)
    return best_model

## Validación del modelo

In [138]:
def validate_model(y_test, predicciones):
    # Impresión de matriz de confusión
    # print("Matriz de confusión:")
    # print(confusion_matrix(Y_test, Predicciones))

    # Impresión de procentaje de Accuracy del modelo
    print("\nAccuracy del modelo: ")
    print(metrics.accuracy_score(y_test, predicciones))

    # Impresión de las métricas para el modelo
    print("\nMétricas de evaluación:")
    print(classification_report(y_test, predicciones))

## Pipeline de todo el proceso

In [139]:
# Cargamos la información y creamos un DataFrame
path = 'DataSet para entrenamiento del modelo.csv'
df = read_corpus(path)

# Preprocesamiento
df_pre = preprocess(df)

# Lectura y split de los datos
X = df_pre['title'].values.tolist()
X = [' '.join(tokens) for tokens in X]
X = tfidf_vect.fit_transform(X)
# X = np.hstack((X.toarray(), df_pre[['text_length', 'lexical_diversity']].values))

y = df_pre['label'].values.tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

model = train_model(X_train, y_train)

# Impresión de métricas
Predicciones = model.predict(X_test)
validate_model(y_test, Predicciones)

Elementos en el DataSet: 16823


## Guardado de modelo

In [ ]:
# Pickle para guardar modelos
import pickle

filename = "model_KikeMau.pickle"

# Guardar el modelo
pickle.dump(model, open(filename, "wb"))


## Prueba del modelo (Parte mas importante)

In [ ]:
def pipeline(input_file, model):
    # Cargamos la información y creamos un DataFrame
    df = read_corpus(path)

    # Preprocesamiento
    df_pre = preprocess(df)

    # Lectura y split de los datos
    X = df_pre['title'].values.tolist()
    y = df_pre['label'].values.tolist()

    X = tfidf_vect.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

    model = train_model(X_train, y_train)

    # Impresión de métricas
    Predicciones = model.predict(X_test)
    validate_model(y_test, Predicciones)


# Prueba para calificación del examen
input_file = 'DataSet para entrenamiento del modelo.csv'
pipeline(input_file, model)

TypeError: Pipeline() missing 1 required positional argument: 'model'